In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)  )  

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time
import seaborn as sns
from matplotlib import pyplot as plt

from warnings import filterwarnings

filterwarnings('ignore')
scaler=MinMaxScaler()

**Readign Datasets ...**

In [ ]:
df_train = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [ ]:
df_train.describe()

What is Outlier?

In statistics, an outlier is a data point that differs significantly from other observations. An outlier may be due to variability in the measurement or it may indicate experimental error; the latter are sometimes excluded from the data set. An outlier can cause serious problems in statistical analyses.

# IQR

First of all, we need to come up with some visualizations. One of the best visuals for this purpose is Boxplot. 

We use **IQR** method so as to detect outliers. IQR tells how spread the middle values are. It can be used to tell when a value is too far from the middle.

![](https://miro.medium.com/max/1400/1*0MPDTLn8KoLApoFvI0P2vQ.png)

* **Q1**: 25% of the data is below this data point.
* **Median**: The central value of the data set. It can also be represented as Q2. 50% of the data is below this data point.
* **Q3**: 75% of the data is below this data point.
* **Minimum**: The data point with the smallest value in the data set that isn’t an outlier.
* **Maximum**: The data point with the biggest value in the data set that isn’t an outlier.
* **IQR**: Represents all the values between Q1 and Q3.

The Min and Max boundaries would be calculated using the following formula:
q1, q3= np.percentile(dataset,[25,75])

iqr = q3 - q1

lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

Now, let's try it for column 'var_6' and see outliers.

In [ ]:
sns.boxplot(df_train['var_6'])
plt.show()

The we implement a function which applies this method over all existing numerical columns in both train and test datasets.

In [ ]:
#IQR
def IQR(df):    
    for col in df.columns:
        if col not in ('ID_code', 'target') :            
            col_new = col + '_IQR'
            df[col_new] = df[col]
            q1, q3= np.percentile(df[col_new],[25,75])
            df[col_new][df[col_new]>=q3] = q3
            df[col_new][df[col_new]<=q1] = q1
    return df
df_train = IQR(df_train)
df_test = IQR(df_test)

# Z-Score

**Z-score** is a statistical measure that tells you how far is a data point from the rest of the dataset. In a more technical term, Z-score tells how many standard deviations away a given observation is from the mean.

Z-score is a parametric measure and it takes two parameters — mean and standard deviation.

![](https://miro.medium.com/max/360/1*DCpleGiHnUbN74hAZozK0w.png)

In [ ]:
def ZSCORE(df):
    for col in df.columns:
        if col not in ('ID_code', 'target') :
            col_zscore = col + "_zscore"
            df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)
    return df
df_train = ZSCORE(df_train)
df_test = ZSCORE(df_test)

# A Different Way!

There is another way to relive the affect of outliers. Imagine the normal data range is between 3 and 25 but there are some values higher than 300. You may simply convert the whole data into 0-1 scale e.g. MinMaxScaller. Then, depending on your scenario, you can add an integer value to all data. Take 0.01 and 0.65 for example. Although, the latter one is much higher, if you add 1 to each of then then you may have 1.01 and 1.65!

![](https://t4.ftcdn.net/jpg/02/61/01/77/360_F_261017702_dQTJU5zXzAT5IBUevKEf7Z3b9T2Oddh6.jpg)